# Import Dependencies

In [15]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 35.8 MB/s eta 0:00:00m eta 0:00:010:0101
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [51]:
import os
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Load Environment

In [33]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name)

In [53]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = np.random.randint(0, env.action_space.n)
        n_state, reward, done, _ = env.step([action])
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:[24.]
Episode:2 Score:[14.]
Episode:3 Score:[14.]
Episode:4 Score:[22.]
Episode:5 Score:[31.]


# Understanding the Environment

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [36]:
env.action_space

Discrete(2)

In [37]:
env.action_space.sample()

0

In [38]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [39]:
env.observation_space.sample()

array([ 6.8627363e-01, -9.6994985e+37, -3.7060118e-01,  1.4258258e+38],
      dtype=float32)

# Train an RL Model

In [40]:
log_path = os.path.join('Training', 'Logs')

In [41]:
# Path for saving training logs
log_path

'Training/Logs'

In [42]:
env = gym.make(environment_name)
# Wrapper for a non-vectorized environment
env = DummyVecEnv([lambda: env])
# Setting model algorithms
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


/Users/fecondon/Documents/DataScience/venviron/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [43]:
PPO??

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    rollout_buffer_class: Optional[Type[stable_baselines3.common.buffers.RolloutBuffer]] = None,
    rollout_buffer_kwargs: Optional[Dict[str, Any]] = None,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[Dict[str, Any]]

In [54]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 7588 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 4870         |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.008228084  |
|    clip_fraction        | 0.0981       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   | -0.021483183 |
|    learning_rate        | 0.0003       |
|    loss                 | 9.67         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0166      |
|    value_loss           | 58.7         |
----------------------------

# Save and Reload Model

In [55]:
ppo_path = os.path.join('Training', 'Saved_Models', 'PPO_Model_Cartpole')

In [56]:
model.save(ppo_path)

/Users/fecondon/Documents/DataScience/venviron/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'Training/Saved_Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [57]:
del model

In [58]:
model = PPO.load(ppo_path, env=env)

# Evaluation

In [59]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/Users/fecondon/Documents/DataScience/venviron/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(np.float64(500.0), np.float64(0.0))

In [60]:
env.close()

# Test Model

In [66]:
action, _ = model.predict(n_state)

In [68]:
action[0]

np.int64(0)

In [69]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        # Using model now
        action = model.predict(obs)
        obs, reward, done, _ = env.step(action[0])
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:[500.]
Episode:2 Score:[245.]
Episode:3 Score:[500.]
Episode:4 Score:[202.]
Episode:5 Score:[208.]


# Viewing Logs in Tensorboard

In [70]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [71]:
training_log_path

'Training/Logs/PPO_2'

In [74]:
!tensorboard --logdir={training_log_path}

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.17.1 at http://localhost:6006/ (Press CTRL+C to quit)
W0828 23:18:11.143083 13321154560 application.py:559] path /data/index.js not found, sending 404
^C


# Adding a Callback to the Training Stage

In [75]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [77]:
save_path = os.path.join('Training', 'Saved_Models')

In [78]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=500, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [79]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [80]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_4
-----------------------------
| time/              |      |
|    fps             | 7627 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 5250          |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.0083532445  |
|    clip_fraction        | 0.108         |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.686        |
|    explained_variance   | -0.0021767616 |
|    learning_rate        | 0.0003        |
|    loss                 | 5.27          |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.0165       |
|    value_loss           | 52.1          |
-----------

# Changing Policies

In [82]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [83]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device


/Users/fecondon/Documents/DataScience/venviron/lib/python3.11/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_5
-----------------------------
| time/              |      |
|    fps             | 6181 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 3815        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015739767 |
|    clip_fraction        | 0.222       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | 0.003305912 |
|    learning_rate        | 0.0003      |
|    loss                 | 2.78        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0248     |
|    value_loss           | 19.2        |
-----------------------------------------
---

# Using an Alternate Algorithm

In [85]:
from stable_baselines3 import DQN

In [86]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [87]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.962    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 7033     |
|    time_elapsed     | 0        |
|    total_timesteps  | 81       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.909    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4811     |
|    time_elapsed     | 0        |
|    total_timesteps  | 191      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.534    |
|    n_updates        | 22       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.877    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 